<h1>Host and Query Model with Grid</h1>
<p>In this series of tutorials we show how you can serve and query models on Grid platform.</p>

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

**NOTE:**
Components:
 - Grid Gateway(http://localhost:5000)
 - Grid Node Alice (http://localhost:3000)
 - Grid Node Bob (http://localhost:3001)
 - Grid Node Bill (http://localhost:3002)

<h3>Import dependencies</h3>

In [1]:
import torch as th
import grid as gr
import grid.syft as sy

# GPT-2 model
from pytorch_transformers import GPT2Tokenizer, GPT2LMHeadModel

ModuleNotFoundError: No module named 'helper'

<h3>Set up configs</h3>
<p>Now, we'll set our syft hook and instantiate our grid</p>

In [ ]:
hook = sy.TorchHook(th)

In [ ]:
my_grid = gr.GridNetwork("http://localhost:5000")

### Load dataset

In [ ]:
df = helper.read_skin_cancer_dataset()
train_df, valid_df, test_df = helper.split_data(df)

# These values are from Part 1.
input_size = 32
train_mean, train_std = (th.tensor([0.6979, 0.5445, 0.5735]), th.tensor([0.0959, 0.1187, 0.1365]))

# Create a test dataloader
test_set = helper.Dataset(test_df, transform=helper.transform(input_size, train_mean, train_std))
test_generator = th.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

# Get a data sample and a target
data, target = next(iter(test_generator))

### Create Model

In [ ]:
# Trace model
model = helper.make_model()
# model.load_state_dict(torch.load("binary-skin-cancer-detection-model"))
model.eval()
traced_model = th.jit.trace(model, data)

# Plan model
plan_model = helper.make_model(is_plan=True)
# model.load_state_dict(torch.load("binary-skin-cancer-detection-model"))
plan_model.eval()
plan_model.build(data)

<h3> Hosting our models</h3>
<p>Now,we'll host our models on grid nodes registered on our grid network. For test purposes, we'll register the same model (can be jit model or plan model) changing only the model_id. </p>  
<p>Nowadays, the choose of grid node that will host the model is random, but in the future we can add a better metric to do this.</p>

In [ ]:
my_grid.serve_model(traced_model, "skin-cancer-grid-model")

<h3>Querying/Running inference</h3>
<p>Now, we want to recover the reference of our models registered previously.</p>
<p>It is important to notice that we can have the same ID on different nodes (we can have different versions of same model distributed throughout our network). To solve this problem, when we perform a query, it will return a list of grid nodes that contains some version of this specific model.</p>

In [ ]:
my_grid.run_remote_inference(model_id="skin-cancer-grid-model", dataset=data)

<h3>Disconnect Nodes</h3>

In [ ]:
my_grid.disconnect_nodes()